### Script para calcular indicadores SFD
El código toma como argumento el script Functions.R en dónde se definen las funciones que calculan todos los indicadores. 
Después se calculan dentro del código general las variables intermedias necesarias para calcular los indicadores y 
finalmente se usa la función de indicadores para las distintas desagregaciones
Autor: María Reyes Retana

In [ ]:
options(warn=-1)
source('utils.R')
source('Functions.R')
options(scipen=999)

In [68]:
# Objetcs
path <- str_c(scldatalake,
              "Household Socio-Economic Surveys/Harmonized Household Surveys/concat/harmonized-latest-v1.csv")
data_total <-s3read_using(FUN = read_delim, 
                      object = path, delim=',')  %>% as_tibble() 

Rows: 1818950 Columns: 164
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (3): pais_c, upm_ci, estrato_ci
dbl (154): region_BID_c, region_c, anio_c, mes_c, zona_c, factor_ch, idh_ch,...
lgl   (7): afroind_ano_ci, rama_ci , tcylmpri_ci, tcylmpri_ch, ppp_wdi2011 ,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [66]:
year = c(2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009) # aquí pones los años que quieres correr y la agarra de las concatenadas. Importante detalle con BRA 2020, 2019

In [ ]:
for (i in year) {
   
    path = c(str_c(scldatalake,
             "Household Socio-Economic Surveys/Harmonized Household Surveys/concat/csv/harmonized-",i,".csv"))
    
    data_total <-s3read_using(FUN = read_csv, 
                      object = path, show_col_types = FALSE)  %>% as_tibble()
    
    data_total <-  data_total %>% 
    select(edusc_ci,edad_ci, jefe_ci, sexo_ci, ylm_ci, ynlm_ci, zona_c, sexo_ci, afroind_ci, dis_ci, edad_ci, anio_c, pais_c, idh_ch, edupc_ci, edusi_ci,
       miembros_ci, lp5_ci, lp31_ci, factor_ci, factor_ch, internet_ch,  cel_ch, aedu_ci, ylnm_ci, ynlnm_ci, nmiembros_ch, eduui_ci, eduuc_ci, edus2c_ci)
    
    gc() # liberar memoria
    
   data_gdi <- data_total %>%
  mutate(age_25_mas = ifelse(edad_ci>=25, 1, 0)) %>%
  mutate(jefa_ci = case_when(jefe_ci==1 & sexo_ci==2 ~ 1, 
                             jefe_ci==1 & sexo_ci==1 ~ 0, 
                             TRUE ~ NA_real_),
         ylm_ci=as.double(ylm_ci), ynlm_ci=as.double(ynlm_ci),
         ylnm_ci=as.double(ylnm_ci),
         urbano_ci = case_when(zona_c == 1 ~ 1, 
                               is.na(zona_c) ~NA_real_, 
                               TRUE ~ 0), 
         pob_sfd = case_when(sexo_ci == 2 ~ 1, 
                             afroind_ci == 1 ~ 1, 
                             afroind_ci == 2 ~ 1, 
                             dis_ci == 1 ~ 1, 
                             TRUE ~ 0), # variable requested for SFD - GDI
         pob18_ci = case_when(edad_ci<=18 ~ 1, 
                              is.na(edad_ci) ~ NA_real_, 
                              TRUE ~ 0), 
         pob65_ci = case_when(edad_ci>=65 ~ 1, 
                              is.na(edad_ci) ~ NA_real_, 
                              TRUE ~ 0),
        pob_indi = ifelse(afroind_ci==1, 1, NA), 
        pob_afro = ifelse(afroind_ci==2, 1, NA)) %>% 
        mutate(age_20_25 = ifelse(edad_ci>=20 & edad_ci <=25, 1, NA),
        age_term_s_c = case_when(pais_c == "BRB" & (edad_ci>=18 & edad_ci<=20) ~ 1, 
                                (pais_c %in% c("COL", "CRI", "GTM", "GTM", "GUY", "JAM", "NIC", "PER", "VEN")) & (edad_ci>=19 & edad_ci<=21) ~ 1,
                                !(pais_c == "COL" | pais_c == "CRI"| pais_c ==  "GTM"| pais_c ==  "GTM"| pais_c ==  "GUY"| pais_c ==  "JAM"
                                  | pais_c == "NIC"| pais_c ==  "PER"| pais_c ==  "VEN"| pais_c ==  "BRB") & (edad_ci>=20 & edad_ci<=22) ~ 1, 
                               TRUE ~ NA_real_), 
        t_cond_sec_aedu = case_when((pais_c == "BRB" | pais_c == "COL" | pais_c == "CRI" |pais_c == "GUY" | pais_c == "PER" | pais_c == "VEN"
                                 | pais_c == "SLV" | pais_c == "GTM" | pais_c == "HND" | pais_c == "JAM" | pais_c == "NIC") 
                                & aedu_ci>=11 & age_term_s_c==1 ~ 1, 
                                (pais_c == "HTI" | pais_c == "SUR") & aedu_ci>=13 & age_term_s_c==1 ~ 1,
                                 aedu_ci>=12 & age_term_s_c==1 & !(pais_c == "BRB" | pais_c == "COL" | pais_c == "CRI" 
                                                                    |pais_c == "GUY" | pais_c == "PER" | pais_c == "VEN"
                                 | pais_c == "SLV" | pais_c == "GTM" | pais_c == "HND" | pais_c == "JAM" | pais_c == "NIC" | 
                                                                  pais_c == "HTI" | pais_c == "SUR") ~ 1,  
                                       TRUE ~ NA_real_),
        t_cond_secundaria = case_when(((edusc_ci==1 | eduui_ci==1 | eduuc_ci==1) & age_term_s_c==1) ~ 1, 
                                     TRUE ~ NA_real_),
        t_cond_secundaria_2025 = case_when((pais_c == "BRB" | pais_c == "COL" | pais_c == "CRI" |pais_c == "GUY" | pais_c == "PER" | pais_c == "VEN"
                                 | pais_c == "SLV" | pais_c == "GTM" | pais_c == "HND" | pais_c == "JAM" | pais_c == "NIC") 
                                & aedu_ci>=11 & age_20_25==1 ~ 1, 
                                (pais_c == "HTI" | pais_c == "SUR") & aedu_ci>=13 & age_20_25==1 ~ 1,
                                 aedu_ci>=12 & age_20_25==1 & !(pais_c == "BRB" | pais_c == "COL" | pais_c == "CRI" 
                                                                    |pais_c == "GUY" | pais_c == "PER" | pais_c == "VEN"
                                 | pais_c == "SLV" | pais_c == "GTM" | pais_c == "HND" | pais_c == "JAM" | pais_c == "NIC" | 
                                                                  pais_c == "HTI" | pais_c == "SUR") ~ 1,  
                                       TRUE ~ NA_real_), 
         t_cond_secundaria_2025_g = case_when(((edusc_ci==1 | eduui_ci==1 | eduuc_ci==1) & age_20_25==1) ~ 1, 
                                     TRUE ~ NA_real_),
         age_term_p_c = case_when(pais_c == "TTO" & (edad_ci>=15 & edad_ci<=17) ~ 1, 
                                (pais_c %in% c("COL", "BRB", "BRA")) & (edad_ci>=13 & edad_ci<=15) ~ 1,
                                !(pais_c == "COL" | pais_c == "BRA"| pais_c ==  "TTO"| pais_c ==  "BRB") & (edad_ci>=14 & edad_ci<=16) ~ 1, 
                               TRUE ~ NA_real_),
        t_cond_prim_aedu = case_when((pais_c == "BRA" | pais_c == "COL") & aedu_ci>=5 & age_term_p_c==1 ~ 1,
                         (pais_c == "TTO") & aedu_ci>=7 & age_term_p_c==1 ~ 1,
                          aedu_ci>=6 & age_term_p_c==1 & !(pais_c == "BRA" | pais_c == "COL" | pais_c == "TTO") ~ 1,
                          TRUE ~NA_real_),
        t_cond_primaria = case_when(((edupc_ci==1 | edusi_ci==1 | edusc_ci==1 | eduui_ci==1 | eduuc_ci==1) & age_term_p_c==1) ~ 1, 
                                     TRUE ~ NA_real_),
        t_cond_primaria_2025 = case_when((pais_c == "BRA" | pais_c == "COL") & aedu_ci>=5 & age_20_25==1 ~ 1,
                         (pais_c == "TTO") & aedu_ci>=7 & age_20_25==1 ~ 1,
                          aedu_ci>=6 & age_20_25==1 & !(pais_c == "BRA" | pais_c == "COL" | pais_c == "TTO") ~ 1,
                          TRUE ~NA_real_)) %>%
  rowwise() %>%  
  # indiv
  mutate(ytot_ci = sum(ylm_ci, ynlm_ci, ynlnm_ci, ylnm_ci, na.rm=TRUE),
         ytot_ci = ifelse(ytot_ci<0, NA, ytot_ci),
         yallsr18 = ifelse(edad_ci>=18, ylm_ci, NA)) %>%
  # fam
  group_by(anio_c, pais_c, idh_ch) %>%
  mutate(ytot_ch = ifelse(miembros_ci==1,sum(ytot_ci,na.rm=TRUE),NA),
         ytot_ch = ifelse(ytot_ch<0, NA, ytot_ch),
         ylm_ch = ifelse(miembros_ci==1,sum(ylm_ci,na.rm=TRUE),NA),
         ylm_ch = ifelse(ylm_ch<0, NA, ylm_ch),
         ylm_women = sum(ifelse(sexo_ci == 2, ylm_ci,NA), na.rm = TRUE),
         ylm_women = ifelse(is.na(ylm_ci), NA, ylm_women),
         ytot_women = sum(ifelse(sexo_ci == 2, ytot_ci,NA), na.rm = TRUE),
         ytot_women = ifelse(is.na(ylm_ci), NA, ytot_women),
         hh_ylm_women = ifelse(ylm_women>0, ylm_women/ylm_ch, # ingreso laboral de la mujer en total laboral del hogar
                              ifelse(ylm_women==0, 0, NA)),
         hh_ytot_women = ifelse(ytot_women>0, ytot_women/ytot_ch, # ingreso total de la mujer en total del hogar
                              ifelse(ytot_women==0, 0, NA))) %>%
ungroup()%>%
 mutate(# Income per cápita definition 
    pc_ytot_ch = ytot_ch/nmiembros_ch,
    pc_ytot_ch = ifelse(pc_ytot_ch<=0,NA,pc_ytot_ch),
    # International poverty
    poor=case_when(pc_ytot_ch <lp5_ci  ~ 1,
                   pc_ytot_ch>=lp5_ci & !is.na(pc_ytot_ch) ~ 0), 
    poor31=case_when(pc_ytot_ch <lp31_ci  ~ 1,
                     pc_ytot_ch>=lp31_ci & !is.na(pc_ytot_ch) ~ 0), 
    poor = ifelse(is.na(pc_ytot_ch),NA,poor),
    poor31 = ifelse(is.na(pc_ytot_ch),NA,poor31))
    
    ##### Average without desagrega
    
    data_average <- data_gdi %>% 
  mutate(sex = "Total", ethnicity = "Total") %>% 
  funcion_indicator_tot(., anio_c, pais_c, sex, ethnicity) 
    
   # By gender 
    
    data_average_gender <- data_gdi %>% 
 mutate(sex = case_when(sexo_ci == 2 ~ "women",
                        sexo_ci == 1 ~"men", 
                          TRUE ~ NA_character_),
          ethnicity = "Total") %>%
 funcion_indicator(., anio_c, pais_c, sex, ethnicity) 
    
   # By ethnicity 
    
  data_average_race <- data_gdi %>% 
 mutate(sex = "Total", 
        ethnicity = case_when(afroind_ci == 1 ~ "Indi", 
                                afroind_ci == 2 ~ "Afro",
                                afroind_ci == 3 ~ "Otro", 
                                is.na(afroind_ci) ~ "Total",
                                afroind_ci == 9 ~ NA_character_)) %>%
  group_by(anio_c, pais_c, sex, ethnicity) %>%
  funcion_indicator(., anio_c, pais_c, sex, ethnicity) %>%
 filter(ethnicity != "Total")  
    
    # by gender and ethnicity
    
    data_average_gender_race <- data_gdi %>% 
 mutate(sex = case_when(sexo_ci == 2 ~ "women",
                          sexo_ci == 1 ~"men", 
                          TRUE ~ NA_character_),
          ethnicity = case_when(afroind_ci == 1 ~ "Indi", 
                                afroind_ci == 2 ~ "Afro",
                                afroind_ci == 3 ~ "Otro", 
                                is.na(afroind_ci) ~ "Total", 
                                afroind_ci == 9 ~ NA_character_)) %>%
  filter(ethnicity != "Total") %>%
  funcion_indicator(., anio_c, pais_c, sex, ethnicity) 
    
   data_total <- do.call("rbind", list(data_average, data_average_gender, data_average_race, data_average_gender_race)) %>%
               mutate(year = anio_c, isoalpha3 = pais_c) %>%
               ungroup() %>%
               select(-c(anio_c, pais_c)) 
    
    # salvar data por año
    
    write.csv(data_total, paste("Outputs/data_total_",i,".csv", sep = ""), row.names = FALSE)
    gc()
    
    
}

#### Uno por uno:
si se quiere correr por base y no en el for de años

In [70]:
path <-  str_c(scldatalake,
              "Household Socio-Economic Surveys/Harmonized Household Surveys/MEX/ENIGH/data_arm/MEX_2008m8_m11_BID.dta")
data_total <-s3read_using(FUN = read_dta, 
                      object = path)  %>% as_tibble() 

In [114]:
data_gdi <- data_total %>%
  mutate(age_25_mas = ifelse(edad_ci>=25, 1, 0)) %>%
  mutate(jefa_ci = case_when(jefe_ci==1 & sexo_ci==2 ~ 1, 
                             jefe_ci==1 & sexo_ci==1 ~ 0, 
                             TRUE ~ NA_real_),
         ylm_ci=as.double(ylm_ci), ynlm_ci=as.double(ynlm_ci),
         ylnm_ci=as.double(ylnm_ci),
         urbano_ci = case_when(zona_c == 1 ~ 1, 
                               is.na(zona_c) ~NA_real_, 
                               TRUE ~ 0), 
         pob_sfd = case_when(sexo_ci == 2 ~ 1, 
                             afroind_ci == 1 ~ 1, 
                             afroind_ci == 2 ~ 1, 
                             dis_ci == 1 ~ 1, 
                             TRUE ~ 0), # variable requested for SFD - GDI
         pob18_ci = case_when(edad_ci<=18 ~ 1, 
                              is.na(edad_ci) ~ NA_real_, 
                              TRUE ~ 0), 
         pob65_ci = case_when(edad_ci>=65 ~ 1, 
                              is.na(edad_ci) ~ NA_real_, 
                              TRUE ~ 0),
        pob_indi = ifelse(afroind_ci==1, 1, NA), 
        pob_afro = ifelse(afroind_ci==2, 1, NA)) %>% 
mutate(age_20_25 = ifelse(edad_ci>=20 & edad_ci <=25, 1, NA),
       age_term_s_c = case_when(pais_c == "BRB" & (edad_ci>=18 & edad_ci<=20) ~ 1, 
                                (pais_c %in% c("COL", "CRI", "GTM", "GTM", "GUY", "JAM", "NIC", "PER", "VEN")) & (edad_ci>=19 & edad_ci<=21) ~ 1,
                                !(pais_c == "COL" | pais_c == "CRI"| pais_c ==  "GTM"| pais_c ==  "GTM"| pais_c ==  "GUY"| pais_c ==  "JAM"
                                  | pais_c == "NIC"| pais_c ==  "PER"| pais_c ==  "VEN"| pais_c ==  "BRB") & (edad_ci>=20 & edad_ci<=22) ~ 1, 
                               TRUE ~ NA_real_), 
       t_cond_sec_aedu = case_when((pais_c == "BRB" | pais_c == "COL" | pais_c == "CRI" |pais_c == "GUY" | pais_c == "PER" | pais_c == "VEN"
                                 | pais_c == "SLV" | pais_c == "GTM" | pais_c == "HND" | pais_c == "JAM" | pais_c == "NIC") 
                                & aedu_ci>=11 & age_term_s_c==1 ~ 1, 
                                (pais_c == "HTI" | pais_c == "SUR") & aedu_ci>=13 & age_term_s_c==1 ~ 1,
                                 aedu_ci>=12 & age_term_s_c==1 & !(pais_c == "BRB" | pais_c == "COL" | pais_c == "CRI" 
                                                                    |pais_c == "GUY" | pais_c == "PER" | pais_c == "VEN"
                                 | pais_c == "SLV" | pais_c == "GTM" | pais_c == "HND" | pais_c == "JAM" | pais_c == "NIC" | 
                                                                  pais_c == "HTI" | pais_c == "SUR") ~ 1,  
                                       TRUE ~ NA_real_),
       t_cond_secundaria = case_when(((edusc_ci==1 | eduui_ci==1 | eduuc_ci==1) & age_term_s_c==1) ~ 1, 
                                     TRUE ~ NA_real_),
       t_cond_secundaria_2025 = case_when((pais_c == "BRB" | pais_c == "COL" | pais_c == "CRI" |pais_c == "GUY" | pais_c == "PER" | pais_c == "VEN"
                                 | pais_c == "SLV" | pais_c == "GTM" | pais_c == "HND" | pais_c == "JAM" | pais_c == "NIC") 
                                & aedu_ci>=11 & age_20_25==1 ~ 1, 
                                (pais_c == "HTI" | pais_c == "SUR") & aedu_ci>=13 & age_20_25==1 ~ 1,
                                 aedu_ci>=12 & age_20_25==1 & !(pais_c == "BRB" | pais_c == "COL" | pais_c == "CRI" 
                                                                    |pais_c == "GUY" | pais_c == "PER" | pais_c == "VEN"
                                 | pais_c == "SLV" | pais_c == "GTM" | pais_c == "HND" | pais_c == "JAM" | pais_c == "NIC" | 
                                                                  pais_c == "HTI" | pais_c == "SUR") ~ 1,  
                                       TRUE ~ NA_real_), 
       t_cond_secundaria_2025_g = case_when(((edusc_ci==1 | eduui_ci==1 | eduuc_ci==1) & age_20_25==1) ~ 1, 
                                     TRUE ~ NA_real_),
       age_term_p_c = case_when(pais_c == "TTO" & (edad_ci>=15 & edad_ci<=17) ~ 1, 
                                (pais_c %in% c("COL", "BRB", "BRA")) & (edad_ci>=13 & edad_ci<=15) ~ 1,
                                !(pais_c == "COL" | pais_c == "BRA"| pais_c ==  "TTO"| pais_c ==  "BRB") & (edad_ci>=14 & edad_ci<=16) ~ 1, 
                               TRUE ~ NA_real_),
       t_cond_prim_aedu = case_when((pais_c == "BRA" | pais_c == "COL") & aedu_ci>=5 & age_term_p_c==1 ~ 1,
                         (pais_c == "TTO") & aedu_ci>=7 & age_term_p_c==1 ~ 1,
                          aedu_ci>=6 & age_term_p_c==1 & !(pais_c == "BRA" | pais_c == "COL" | pais_c == "TTO") ~ 1,
                          TRUE ~NA_real_),
       t_cond_primaria = case_when(((edupc_ci==1 | edusi_ci==1 | edusc_ci==1 | eduui_ci==1 | eduuc_ci==1) & age_term_p_c==1) ~ 1, 
                                     TRUE ~ NA_real_),
       t_cond_primaria_2025 = case_when((pais_c == "BRA" | pais_c == "COL") & aedu_ci>=5 & age_20_25==1 ~ 1,
                         (pais_c == "TTO") & aedu_ci>=7 & age_20_25==1 ~ 1,
                          aedu_ci>=6 & age_20_25==1 & !(pais_c == "BRA" | pais_c == "COL" | pais_c == "TTO") ~ 1,
                          TRUE ~NA_real_)) %>%
  rowwise() %>%  
  # indiv
  mutate(ytot_ci = sum(ylm_ci, ynlm_ci, ynlnm_ci, ylnm_ci, na.rm=TRUE),
         ytot_ci = ifelse(ytot_ci<0, NA, ytot_ci),
         yallsr18 = ifelse(edad_ci>=18, ylm_ci, NA)) %>%
  # fam
  group_by(anio_c, pais_c, idh_ch) %>%
  mutate(ytot_ch = ifelse(miembros_ci==1,sum(ytot_ci,na.rm=TRUE),NA),
         ytot_ch = ifelse(ytot_ch<0, NA, ytot_ch),
         ylm_ch = ifelse(miembros_ci==1,sum(ylm_ci,na.rm=TRUE),NA),
         ylm_ch = ifelse(ylm_ch<0, NA, ylm_ch),
         ylm_women = sum(ifelse(sexo_ci == 2, ylm_ci,NA), na.rm = TRUE),
         ylm_women = ifelse(is.na(ylm_ci), NA, ylm_women),
         ytot_women = sum(ifelse(sexo_ci == 2, ytot_ci,NA), na.rm = TRUE),
         ytot_women = ifelse(is.na(ylm_ci), NA, ytot_women),
         hh_ylm_women = ifelse(ylm_women>0, ylm_women/ylm_ch, # ingreso laboral de la mujer en total laboral del hogar
                              ifelse(ylm_women==0, 0, NA)),
         hh_ytot_women = ifelse(ytot_women>0, ytot_women/ytot_ch, # ingreso total de la mujer en total del hogar
                              ifelse(ytot_women==0, 0, NA))) %>%
ungroup()%>%
 mutate(# Income per cápita definition 
    pc_ytot_ch = ytot_ch/nmiembros_ch,
    pc_ytot_ch = ifelse(pc_ytot_ch<=0,NA,pc_ytot_ch),
    # International poverty
    poor=case_when(pc_ytot_ch <lp5_ci  ~ 1,
                   pc_ytot_ch>=lp5_ci & !is.na(pc_ytot_ch) ~ 0), 
    poor31=case_when(pc_ytot_ch <lp31_ci  ~ 1,
                     pc_ytot_ch>=lp31_ci & !is.na(pc_ytot_ch) ~ 0), 
    poor = ifelse(is.na(pc_ytot_ch),NA,poor),
    poor31 = ifelse(is.na(pc_ytot_ch),NA,poor31))
 

#### Average: without disagregation

In [ ]:
data_average <- data_gdi %>% 
  mutate(sex = "Total", ethnicity = "Total") %>% 
  funcion_indicator_tot(., anio_c, pais_c, sex, ethnicity) 

### Average: by gender

In [101]:
data_average_gender <- data_gdi %>% 
 mutate(sex = case_when(sexo_ci == 2 ~ "women",
                        sexo_ci == 1 ~"men", 
                          TRUE ~ NA_character_),
          ethnicity = "Total") %>%
 funcion_indicator(., anio_c, pais_c, sex, ethnicity) 

`summarise()` has grouped output by 'anio_c', 'pais_c', 'sex'. You can override
using the `.groups` argument.


### Average: by race

In [103]:
data_average_race <- data_gdi %>% 
 mutate(sex = "Total", 
        ethnicity = case_when(afroind_ci == 1 ~ "Indi", 
                                afroind_ci == 2 ~ "Afro",
                                afroind_ci == 3 ~ "Otro", 
                                is.na(afroind_ci) ~ "Total",
                                afroind_ci == 9 ~ NA_character_)) %>%
  group_by(anio_c, pais_c, sex, ethnicity) %>%
  funcion_indicator(., anio_c, pais_c, sex, ethnicity) %>%
 filter(ethnicity != "Total") 

`summarise()` has grouped output by 'anio_c', 'pais_c', 'sex'. You can override
using the `.groups` argument.


### Average: by gender and race

In [104]:
data_average_gender_race <- data_gdi %>% 
 mutate(sex = case_when(sexo_ci == 2 ~ "women",
                          sexo_ci == 1 ~"men", 
                          TRUE ~ NA_character_),
          ethnicity = case_when(afroind_ci == 1 ~ "Indi", 
                                afroind_ci == 2 ~ "Afro",
                                afroind_ci == 3 ~ "Otro", 
                                is.na(afroind_ci) ~ "Total", 
                                afroind_ci == 9 ~ NA_character_)) %>%
  filter(ethnicity != "Total") %>%
  funcion_indicator(., anio_c, pais_c, sex, ethnicity) 

`summarise()` has grouped output by 'anio_c', 'pais_c', 'sex'. You can override
using the `.groups` argument.


### All

In [105]:
data_total <- do.call("rbind", list(data_average, data_average_gender, data_average_race, data_average_gender_race)) %>%
               mutate(year = anio_c, isoalpha3 = pais_c) %>%
               ungroup() %>%
               select(-c(anio_c, pais_c))

In [ ]:
write.csv(data_total, paste("Outputs/data_total_",year,".csv", sep = ""), row.names = FALSE)